In [ ]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot
from infrastructure.instrument_collection import instrumentCollection as ic



In [49]:
pair = "EUR_USD"
granularity = "H1"
df = pd.read_pickle(f"./data/{pair}_{granularity}.pkl")
MA_LIST = [10,20,50,100,200]


In [50]:
df.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
3995,2025-10-22 19:00:00+00:00,4048,1.16092,1.16112,1.16048,1.16060,1.16083,1.16104,1.16041,1.16052,1.16100,1.16121,1.16056,1.16067
3996,2025-10-22 20:00:00+00:00,1576,1.16059,1.16110,1.16058,1.16100,1.16051,1.16101,1.16051,1.16091,1.16067,1.16118,1.16066,1.16109
3997,2025-10-22 21:00:00+00:00,573,1.16124,1.16134,1.16106,1.16130,1.16083,1.16115,1.16067,1.16111,1.16165,1.16165,1.16124,1.16150
3998,2025-10-22 22:00:00+00:00,839,1.16130,1.16146,1.16112,1.16123,1.16112,1.16138,1.16102,1.16114,1.16148,1.16155,1.16122,1.16132
3999,2025-10-22 23:00:00+00:00,2142,1.16125,1.16127,1.16067,1.16072,1.16117,1.16118,1.16059,1.16063,1.16133,1.16137,1.16075,1.16081


In [51]:
# turn dates type to string to avoid having weekend gaps on chart
# this is a workaround, better solution would be to use rangebreaks in plotly
df['sTime']=[dt.datetime.strftime(x,"s%y-%m-%d %H:%M") for x in df.time] 

In [52]:

df_plot = df.iloc[:200] # plot only first 200 rows for clarity

In [53]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.sTime,
    open=df_plot.mid_o,
     high=df_plot.mid_h,
     low=df_plot.mid_l,
     close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C",
))
fig.update_yaxes(
    gridcolor="#1f292f"
)
fig.update_xaxes(
    gridcolor="#1f292f",
    rangeslider=dict(visible=False),
    nticks=5
)
fig.update_layout(
    width=900,
    height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    paper_bgcolor="#2c303c",
    plot_bgcolor="#2c303c",
    font=dict(size=8, color="#e1e1e1"),
)
fig.show()

In [54]:
df['time']

0      2025-03-04 09:00:00+00:00
1      2025-03-04 10:00:00+00:00
2      2025-03-04 11:00:00+00:00
3      2025-03-04 12:00:00+00:00
4      2025-03-04 13:00:00+00:00
                  ...           
3995   2025-10-22 19:00:00+00:00
3996   2025-10-22 20:00:00+00:00
3997   2025-10-22 21:00:00+00:00
3998   2025-10-22 22:00:00+00:00
3999   2025-10-22 23:00:00+00:00
Name: time, Length: 4000, dtype: datetime64[ns, tzutc()]

In [55]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2025-03-04 09:00:00+00:00,1.05188,1.05275,1.05094,1.05122
1,2025-03-04 10:00:00+00:00,1.05120,1.05196,1.05026,1.05114
2,2025-03-04 11:00:00+00:00,1.05113,1.05447,1.05096,1.05442
3,2025-03-04 12:00:00+00:00,1.05444,1.05571,1.05361,1.05532
4,2025-03-04 13:00:00+00:00,1.05533,1.05595,1.05308,1.05391


In [56]:
for ma in MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()


In [57]:
df_ma.dropna(inplace=True) # remove rows with NaN values
df_ma.reset_index(drop=True, inplace=True) # reset index after dropping rows
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,MA_50,MA_100,MA_200
0,2025-03-14 15:00:00+00:00,1.08779,1.08868,1.08669,1.08715,1.086947,1.085877,1.087088,1.087727,1.081731
1,2025-03-14 16:00:00+00:00,1.08716,1.08900,1.08704,1.08831,1.087391,1.086030,1.087028,1.087766,1.081917
2,2025-03-14 17:00:00+00:00,1.08832,1.08881,1.08791,1.08824,1.087821,1.086179,1.086944,1.087813,1.082102
3,2025-03-14 18:00:00+00:00,1.08823,1.08864,1.08734,1.08802,1.088085,1.086317,1.086893,1.087848,1.082270
4,2025-03-14 19:00:00+00:00,1.08802,1.08846,1.08784,1.08831,1.088392,1.086457,1.086845,1.087894,1.082435


In [58]:





df_plot = df_ma.iloc[:500] # plot only first 100 rows for clarity
cp = CandlePlot(df_plot)
# cp.fig.add_trace(go.Scatter(
#     x=cp.df_plot.sTime,
#     y=cp.df_plot.MA_10,
#     line=dict(color='#FFD700', width=2),
#     name='MA 10',
#     line_shape='spline',
# ))
traces = [f"MA_{ma}" for ma in MA_LIST]
cp.show_plot(line_traces=traces)

In [59]:
MA_SHORT = "MA_10"
MA_LONG = "MA_20"
BUY = 1
SELL = -1
NONE = 0
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_SHORT, MA_LONG]].copy()
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20
0,2025-03-14 15:00:00+00:00,1.08779,1.08868,1.08669,1.08715,1.086947,1.085877
1,2025-03-14 16:00:00+00:00,1.08716,1.08900,1.08704,1.08831,1.087391,1.086030
2,2025-03-14 17:00:00+00:00,1.08832,1.08881,1.08791,1.08824,1.087821,1.086179
3,2025-03-14 18:00:00+00:00,1.08823,1.08864,1.08734,1.08802,1.088085,1.086317
4,2025-03-14 19:00:00+00:00,1.08802,1.08846,1.08784,1.08831,1.088392,1.086457


In [60]:
df_an['DELTA'] = df_an.MA_10 - df_an.MA_20
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)
df_an

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,DELTA,DELTA_PREV
0,2025-03-14 15:00:00+00:00,1.08779,1.08868,1.08669,1.08715,1.086947,1.085877,0.001070,NaN
1,2025-03-14 16:00:00+00:00,1.08716,1.08900,1.08704,1.08831,1.087391,1.086030,0.001361,0.001070
2,2025-03-14 17:00:00+00:00,1.08832,1.08881,1.08791,1.08824,1.087821,1.086179,0.001642,0.001361
3,2025-03-14 18:00:00+00:00,1.08823,1.08864,1.08734,1.08802,1.088085,1.086317,0.001768,0.001642
4,2025-03-14 19:00:00+00:00,1.08802,1.08846,1.08784,1.08831,1.088392,1.086457,0.001935,0.001768
...,...,...,...,...,...,...,...,...,...
3796,2025-10-22 19:00:00+00:00,1.16092,1.16112,1.16048,1.16060,1.160180,1.160344,-0.000164,-0.000246
3797,2025-10-22 20:00:00+00:00,1.16059,1.16110,1.16058,1.16100,1.160422,1.160371,0.000051,-0.000164
3798,2025-10-22 21:00:00+00:00,1.16124,1.16134,1.16106,1.16130,1.160727,1.160411,0.000316,0.000051
3799,2025-10-22 22:00:00+00:00,1.16130,1.16146,1.16112,1.16123,1.161020,1.160430,0.000591,0.000316


In [61]:
def isTrade(row):
    if row.DELTA > 0 and row.DELTA_PREV <= 0:
        return BUY
    elif row.DELTA < 0 and row.DELTA_PREV >= 0:
        return SELL
    else:
        return NONE

In [62]:
df_an['Signal'] = df_an.apply(isTrade, axis=1)
df_an

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,DELTA,DELTA_PREV,Signal
0,2025-03-14 15:00:00+00:00,1.08779,1.08868,1.08669,1.08715,1.086947,1.085877,0.001070,NaN,0
1,2025-03-14 16:00:00+00:00,1.08716,1.08900,1.08704,1.08831,1.087391,1.086030,0.001361,0.001070,0
2,2025-03-14 17:00:00+00:00,1.08832,1.08881,1.08791,1.08824,1.087821,1.086179,0.001642,0.001361,0
3,2025-03-14 18:00:00+00:00,1.08823,1.08864,1.08734,1.08802,1.088085,1.086317,0.001768,0.001642,0
4,2025-03-14 19:00:00+00:00,1.08802,1.08846,1.08784,1.08831,1.088392,1.086457,0.001935,0.001768,0
...,...,...,...,...,...,...,...,...,...,...
3796,2025-10-22 19:00:00+00:00,1.16092,1.16112,1.16048,1.16060,1.160180,1.160344,-0.000164,-0.000246,0
3797,2025-10-22 20:00:00+00:00,1.16059,1.16110,1.16058,1.16100,1.160422,1.160371,0.000051,-0.000164,1
3798,2025-10-22 21:00:00+00:00,1.16124,1.16134,1.16106,1.16130,1.160727,1.160411,0.000316,0.000051,0
3799,2025-10-22 22:00:00+00:00,1.16130,1.16146,1.16112,1.16123,1.161020,1.160430,0.000591,0.000316,0
